In [44]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes

def hybrid_rsa_aes_encryption(data, public_key):
    # Generate a random AES key
    aes_key = get_random_bytes(16)

    # Encrypt the data with AES
    cipher_aes = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(data)

    # Encrypt the AES key with RSA
    cipher_rsa = PKCS1_OAEP.new(public_key)
    cipher_key = cipher_rsa.encrypt(aes_key)

    return (ciphertext, tag, cipher_aes.nonce, cipher_key)

def hybrid_rsa_aes_decryption(ciphertext, tag, nonce, cipher_key, private_key):
    # Decrypt the AES key with RSA
    cipher_rsa = PKCS1_OAEP.new(private_key)
    aes_key = cipher_rsa.decrypt(cipher_key)

    # Decrypt the data with AES
    cipher_aes = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
    data = cipher_aes.decrypt_and_verify(ciphertext, tag)

    return data


In [45]:
from Crypto.PublicKey import RSA

# Generate RSA key pair for demonstration purposes
key = RSA.generate(2048)
public_key = key.publickey()
private_key = key

# Original data to be encrypted
original_data = b"This is a confidential message."

# Encryption using the hybrid_rsa_aes_encryption function
encrypted_data = hybrid_rsa_aes_encryption(original_data, public_key)

# Decryption using the hybrid_rsa_aes_decryption function
decrypted_data = hybrid_rsa_aes_decryption(*encrypted_data, private_key)

# Print the results
print("Original Data:", original_data)
print("Encrypted Data:", encrypted_data)
print("Decrypted Data:", decrypted_data)

# Verify if the decryption is successful
assert original_data == decrypted_data, "Decryption failed!"
print("Decryption successful.")


Original Data: b'This is a confidential message.'
Encrypted Data: (b'\xe4F\x00\xf2\x9c\\Pw>\\\x0c\x84\xac\xf9\xd0\xa3\x8c\xa5\x86+J\xd1yO)\xb7\xffOi&\xbe', b'u\xdf\xbe?\x00\x94o\x92<D&-\xcc!\xf0\xe6', b'\x94K)h!\x9bU\x95E\xb7\xba\xd3E\xb99\xde', b'E,r\xc8XZ\x92W\xdf-\x1f\xcb\xa2^4>\xf6\xa6\x169\x0e\x1fPUB\xe9xNHO\xadhL\xd1\xbd\\\x1b\x10\x12\xd9KU\xd9\x1c\x1f\n\x87\xe4c\xd4\x0b\xa0\xab\xcb\xa1\xae\x11\xc7\x01\x1b\xbb\x1b\xd4d=\xe2\x0f\xcb\xd9\xad\xf09\x9dB\x98\x83n\xe8C\xfa4G\xf5=\xca\xb2%\x821\x1b\xf8\xce\xa93\x08\xea\xa3#\x8f\x07\xdb\x0f\xdb\xc0A\xa1P\\\x8c\x99Lw\x10q\xf1\x08\xd6\x00+-.P\xaa\xef\x94\x1at\x87 \xaa\xaf\x7f\x0eK\xc4\xef\xc74\xad\xb6\xf8a\xa1 \x9d\xd3\x0b\x16\x7f\xe4\x8d\x1b\x96\xecP\x85\xdeU\xfb\xc2\xd2 ]\xcb;[\xfb#\x8aa{[\x99\r\xa9#\x9a\x17X\x98=\xd9\xb2\xec:a\x7f@\xe1m\x0b\xfb\x0cj\xf9\x0b\xf6!\x8dGy`\xaai\x84\xa5o\x9dRB\xb5\xcb\x1f\x93p\xf9\x83b\x163\xd8=U\x10\xddHP\xc0\x83l\x0f\xf2\xe7=?\xeb\xf3.\xe6x\xb0\xd0\xb8\xf2{\t\xd0\x86r\xa7\xda2\xd7\xf2\xdb\xe5')
Decrypted D